In [1]:
import load_files as lf
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import seaborn as sns
import measurement_analysis as ma
import pickle
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [3]:
file_path = "/Users/blazejmanczak/Desktop/Q1/DataChallenge3/"
flow_data_DRU, level_data_DRU = lf.get_measurements(file_path + "waterschap-aa-en-maas_sewage_2019/sewer_data/data_pump/RG8150")
rain_data = lf.get_rain(file_path +"/waterschap-aa-en-maas_sewage_2019/sewer_data/rain_timeseries")
data = lf.sdf(file_path + "waterschap-aa-en-maas_sewage_2019/sewer_model/aa-en-maas_sewer_shp")

In [4]:
# Either run this cell or open pickle in the next one
#df = ma.measurement_analysis(flow_data_DRU, level_data_DRU, rain_data, area_data=data.area_data,
 #                            village_code="DRU", dry_threshold = 2.5)

KeyError: 'DrySeries'

In [9]:
def summarize_rain_data(rain_data, area_data=None, village_code=None, dry_threshold=2.5):
    """
    Function to reshape rain data to be fit for the DWAAS analysis.

    ~~~~~ INPUT  ~~~~~
    rain_data:     File as gathered by load_files.get_rain(...)
    area_data:     File as gathered by load_files.sdf(...).area_data
    village_code:  Identifier of the pump (e.g. 'DRU' for Drunen)
    dry_threshold: Minimum average rain per hour in the area that counts as wet
                   (2.5 recommended)

    ~~~~~ OUTPUT ~~~~~
    A data frame with the columns
    Date :     Date of measurement
    Total:     Average rainfall measurement in the area (unweighted by area size)
    DrySeries: Number of days since last rainfall.
    """

    # Convert to datetime if necessary
    if rain_data["Start"].dtype != "<M8[ns]":
            rain_data["Start"] = pd.to_datetime(rain_data["Start"])

    # Sort data by time because of it being possibly unordered
    rain_data.sort_values("Start", inplace=True)
    rain_data.reset_index(drop=True, inplace=True)

    # Selects only data from certain right village_code
    if village_code is not None:
        area_data["village_ID"] = area_data["sewer_system"].str.slice(4,7)
        area_data = area_data.loc[area_data["village_ID"] == village_code]
        areas = area_data["area_name"][area_data["area_name"].apply(lambda i: i in rain_data.columns)].to_list()

        rain_data = rain_data.loc[:, ["Start", "End"] + areas]

    # Create date column and sum up rain measurements over all area
    rain_data["Date"] = rain_data["Start"].apply(lambda i: i.date())
    rain_data["Total"] = rain_data.iloc[:, 2:].mean(axis=1)

    # Sum measurements by date and create dry-series column
    #rain_data = rain_data.groupby("Date")["Total"].sum().reset_index(drop=False)
    #rain_data["DrySeries"] = reset_cumsum(rain_data["Total"], dry_threshold)

    return rain_data


In [10]:
"""
#Save pickle
filename = 'ma_df'
outfile = open(filename,'wb')
pickle.dump(df,outfile)
outfile.close()
"""

# open pickle
pickle_in = open("/Users/blazejmanczak/Desktop/Q1/DataChallenge3/ma_df", "rb")
df = pickle.load(pickle_in)


In [13]:
rain_data_DRU = summarize_rain_data(rain_data, area_data= df.area_data, village_code="DRU", dry_threshold=2.5)[['Start', 'End', 'Total']]

In [17]:
rain_data_DRU.to_csv("rainDru.csv")

In [18]:
level_data_DRU.to_csv("levelDru.csv")

In [8]:
rain_data = df.rain_data.copy()
rain_data.set_index('Date', inplace = True)

In [9]:
leve

,Total,DrySeries
Date,,
2017-12-31,0.000000,0
2018-01-01,9.104127,0
2018-01-02,4.228682,0
2018-01-03,0.000000,1
2018-01-04,1.026882,2
...,...,...
2019-12-03,3.528955,0
2019-12-04,0.000000,1
2019-12-05,0.000000,2
